### Base de Datos : Citi Bike

Citi Bike es un sistema de intercambio de bicicletas públicas de gestión privada que sirve en parte de la ciudad de Nueva York. Se dedidío trabajar cone sta base de datos para la optimización de ingresos dependiendo del tipo de pase que ofrece la compañía. 

#### Diccionario de datos:

+ Trip Duration (seconds)
+ Start Time and Date
+ Stop Time and Date
+ Start Station Name
+ End Station Name
+ Station ID
+ Station Lat/Long
+ Bike ID
+ User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member)
+ Gender (Zero=unknown; 1=male; 2=female)
+ Year of Birth

Para tener la base de datos lista hicimos algunos supuestos para completarla de acuerdo a la información que necesitamos para modelarla, se crearan las siguientes variables:
+ Un id_trip - único por viaje
+ Una columna que desagrege *User Type* para tener los 3 tipos de pases que ofrece la compañia
+ Una columna que se llama *trip_category* para obtener si el viaje fue : *one way o round trip*

Estas columnas se crearon respetando el modelo del negocio y con la información de sus reportes mensuales, en este caso de Diciembre 2019.

In [1]:
import pandas as pd

data = pd.read_csv('../data/201912-citibike-tripdata.csv.zip')
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,602,2019-12-01 00:00:05.5640,2019-12-01 00:10:07.8180,3382,Carroll St & Smith St,40.680611,-73.994758,3304,6 Ave & 9 St,40.668127,-73.983776,41932,Subscriber,1970,1
1,1206,2019-12-01 00:00:10.9630,2019-12-01 00:20:17.8820,362,Broadway & W 37 St,40.751726,-73.987535,500,Broadway & W 51 St,40.762288,-73.983362,18869,Customer,1999,1
2,723,2019-12-01 00:00:11.8180,2019-12-01 00:12:14.8310,146,Hudson St & Reade St,40.716250,-74.009106,238,Bank St & Washington St,40.736197,-74.008592,15334,Subscriber,1997,1
3,404,2019-12-01 00:00:12.2200,2019-12-01 00:06:56.8860,3834,Irving Ave & Halsey St,40.694670,-73.906630,3827,Halsey St & Broadway,40.685650,-73.915640,41692,Customer,1995,1
4,1059,2019-12-01 00:00:14.7230,2019-12-01 00:17:54.1860,500,Broadway & W 51 St,40.762288,-73.983362,3323,W 106 St & Central Park West,40.798186,-73.960591,40156,Subscriber,1961,1


In [2]:
#Creación de la columna trip_id

#generamos los ID's
n = len(data.index)
digits_range = range(1345,n + 1345)
digits_list = list(digits_range)

#Agreagamos la columna al df
data['trip_id'] = digits_list

In [3]:
#Se obtienen las filas que tienen como usertype "Customer"
subset_c = data.loc[data['usertype'] == 'Customer']

#Se obtienen las filas que tienen como usertype "Subscriber"
subset_s = data.loc[data['usertype'] == 'Subscriber']

In [4]:
data['usertype'].unique()

array(['Subscriber', 'Customer'], dtype=object)

Esta parte es la que se tiene que dividir en 2 tipo de costumer:
+ costumer a =  single-day passes
+ costumer b = single trip

De acuerdo a lo obtenido en el reporte de ese mes de CitiBike : https://d21xlh2maitm24.cloudfront.net/nyc/December-2019-Citi-Bike-Monthly-Report.pdf?mtime=20200218125724
Se tiene que en el 2019(12) en promedio tuvieron estos pases en cada una de las modalidades que manejan:
+ 43,395 single-trip (68.4%)
+ 10,816 one-day-pass (31.6%)

In [5]:
subset_c.shape[0]

#Número de pases de 1 día
single_trip = round(subset_c.shape[0]*.684)

#Número de pases de 3 días
one_day = subset_c.shape[0] - single_trip

print("Numero de pases en diciembre de 1 día",one_day )
print("Numero de pases en diciembre de un viaje sencillo",single_trip )
one_day + single_trip

Numero de pases en diciembre de 1 día 24529
Numero de pases en diciembre de un viaje sencillo 53096


77625

In [6]:
import numpy as np

#Creación de columna auxiliar para separar los pases de Costumer en 2
subset_c['aux']=np.random.binomial(1,.684,77625)

<ipython-input-6-2694fef72fb7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_c['aux']=np.random.binomial(1,.684,77625)


In [7]:
#Creación de columna con el nombre de los pases nuevos
subset_c['type_pass'] = np.where(subset_c['aux'] == 1, "single_trip", "one_day")
#Borramos la columa auxiliar
del subset_c['aux']

subset_s['type_pass']='annual'
#Juntamos los dos dataframes
data = subset_s.append(subset_c, ignore_index=True)
data.shape

<ipython-input-7-5535b8e5e623>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_c['type_pass'] = np.where(subset_c['aux'] == 1, "single_trip", "one_day")
<ipython-input-7-5535b8e5e623>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_s['type_pass']='annual'


(955210, 17)

In [8]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,trip_id,type_pass
0,602,2019-12-01 00:00:05.5640,2019-12-01 00:10:07.8180,3382,Carroll St & Smith St,40.680611,-73.994758,3304,6 Ave & 9 St,40.668127,-73.983776,41932,Subscriber,1970,1,1345,annual
1,723,2019-12-01 00:00:11.8180,2019-12-01 00:12:14.8310,146,Hudson St & Reade St,40.716250,-74.009106,238,Bank St & Washington St,40.736197,-74.008592,15334,Subscriber,1997,1,1347,annual
2,1059,2019-12-01 00:00:14.7230,2019-12-01 00:17:54.1860,500,Broadway & W 51 St,40.762288,-73.983362,3323,W 106 St & Central Park West,40.798186,-73.960591,40156,Subscriber,1961,1,1349,annual
3,1257,2019-12-01 00:00:20.3490,2019-12-01 00:21:18.2990,340,Madison St & Clinton St,40.712690,-73.987763,340,Madison St & Clinton St,40.712690,-73.987763,40829,Subscriber,1994,1,1350,annual
4,624,2019-12-01 00:00:22.0410,2019-12-01 00:10:46.4270,469,Broadway & W 53 St,40.763441,-73.982681,526,E 33 St & 5 Ave,40.747659,-73.984907,24934,Subscriber,1983,1,1351,annual


In [9]:
data['type_pass'].unique()

array(['annual', 'single_trip', 'one_day'], dtype=object)

In [11]:
#Se obtienen las filas que tienen como usertype "Customer"
subset_s = data.loc[data['type_pass'] == 'single_trip']

subset_s['trip_category']='one-way'

subset_o = data.drop(subset_s.index)

<ipython-input-11-8eb34e9bdbd4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_s['trip_category']='one-way'


In [12]:
subset_o.shape

(902063, 17)

In [13]:
#Creación de una columna nueva : trip_category
subset_o['aux']=np.random.binomial(1,.192,902063)

subset_o['trip_category'] = np.where(subset_o['aux'] == 1, "round-trip", "one-way")
#Borramos la columa auxiliar
del subset_o['aux']
subset_o.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,trip_id,type_pass,trip_category
0,602,2019-12-01 00:00:05.5640,2019-12-01 00:10:07.8180,3382,Carroll St & Smith St,40.680611,-73.994758,3304,6 Ave & 9 St,40.668127,-73.983776,41932,Subscriber,1970,1,1345,annual,round-trip
1,723,2019-12-01 00:00:11.8180,2019-12-01 00:12:14.8310,146,Hudson St & Reade St,40.716250,-74.009106,238,Bank St & Washington St,40.736197,-74.008592,15334,Subscriber,1997,1,1347,annual,one-way
2,1059,2019-12-01 00:00:14.7230,2019-12-01 00:17:54.1860,500,Broadway & W 51 St,40.762288,-73.983362,3323,W 106 St & Central Park West,40.798186,-73.960591,40156,Subscriber,1961,1,1349,annual,one-way
3,1257,2019-12-01 00:00:20.3490,2019-12-01 00:21:18.2990,340,Madison St & Clinton St,40.712690,-73.987763,340,Madison St & Clinton St,40.712690,-73.987763,40829,Subscriber,1994,1,1350,annual,one-way
4,624,2019-12-01 00:00:22.0410,2019-12-01 00:10:46.4270,469,Broadway & W 53 St,40.763441,-73.982681,526,E 33 St & 5 Ave,40.747659,-73.984907,24934,Subscriber,1983,1,1351,annual,one-way


In [14]:
#Juntamos los dos dataframes
data = subset_s.append(subset_o, ignore_index=True)
data.shape

(955210, 18)

In [15]:
data

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,trip_id,type_pass,trip_category
0,1206,2019-12-01 00:00:10.9630,2019-12-01 00:20:17.8820,362,Broadway & W 37 St,40.751726,-73.987535,500,Broadway & W 51 St,40.762288,-73.983362,18869,Customer,1999,1,1346,single_trip,one-way
1,1223,2019-12-01 00:00:39.7930,2019-12-01 00:21:03.6680,465,Broadway & W 41 St,40.755136,-73.986580,359,E 47 St & Park Ave,40.755103,-73.974987,16810,Customer,1982,1,1354,single_trip,one-way
2,1570,2019-12-01 00:01:57.0540,2019-12-01 00:28:07.7410,309,Murray St & West St,40.714979,-74.013012,514,12 Ave & W 40 St,40.760875,-74.002777,41997,Customer,1973,1,1364,single_trip,one-way
3,4027,2019-12-01 00:03:25.4140,2019-12-01 01:10:32.6300,3053,Marcy Ave & Lafayette Ave,40.690081,-73.947915,3053,Marcy Ave & Lafayette Ave,40.690081,-73.947915,28207,Customer,1994,1,1375,single_trip,one-way
4,316,2019-12-01 00:06:21.1090,2019-12-01 00:11:37.3090,361,Allen St & Hester St,40.716059,-73.991908,307,Canal St & Rutgers St,40.714275,-73.989900,21186,Customer,1975,1,1388,single_trip,one-way
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955205,1426,2019-12-31 23:49:25.2490,2020-01-01 00:13:11.9430,167,E 39 St & 3 Ave,40.748901,-73.976049,3680,E 43 St & 5 Ave,40.754121,-73.980252,15808,Customer,1969,0,956517,one_day,round-trip
955206,495,2019-12-31 23:52:57.0460,2020-01-01 00:01:12.6610,3151,E 81 St & York Ave,40.772838,-73.949892,3362,Madison Ave & E 82 St,40.778131,-73.960694,39476,Customer,1987,2,956535,one_day,round-trip
955207,484,2019-12-31 23:53:13.2270,2020-01-01 00:01:17.6570,3151,E 81 St & York Ave,40.772838,-73.949892,3362,Madison Ave & E 82 St,40.778131,-73.960694,41085,Customer,1969,0,956538,one_day,round-trip
955208,1775,2019-12-31 23:55:21.1960,2020-01-01 00:24:56.6520,3306,10 St & 7 Ave,40.666208,-73.981999,3416,7 Ave & Park Pl,40.677615,-73.973243,35047,Customer,1981,2,956547,one_day,round-trip


Para la creación de la columna *trip_category* tomamos como referencia el sistema de bicis compartidas de LA metro Bike Share.

In [16]:
#Convertimos la duración a minutos
data['duration'] = round(data['tripduration']/60)

In [17]:
data['starttime'] = pd.to_datetime(data['starttime']).dt.strftime('%Y-%m-%d %H:%M')
data['stoptime'] = pd.to_datetime(data['stoptime']).dt.strftime('%Y-%m-%d %H:%M')

In [36]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,trip_id,type_pass,trip_category,duration
0,1206,2019-12-01 00:00,2019-12-01 00:20,362,Broadway & W 37 St,40.751726,-73.987535,500,Broadway & W 51 St,40.762288,-73.983362,18869,Customer,1999,1,1346,single_trip,one-way,20.0
1,404,2019-12-01 00:00,2019-12-01 00:06,3834,Irving Ave & Halsey St,40.694670,-73.906630,3827,Halsey St & Broadway,40.685650,-73.915640,41692,Customer,1995,1,1348,single_trip,one-way,7.0
2,4027,2019-12-01 00:03,2019-12-01 01:10,3053,Marcy Ave & Lafayette Ave,40.690081,-73.947915,3053,Marcy Ave & Lafayette Ave,40.690081,-73.947915,28207,Customer,1994,1,1375,single_trip,one-way,67.0
3,448,2019-12-01 00:04,2019-12-01 00:11,3332,Degraw St & Hoyt St,40.681990,-73.990790,392,Jay St & Tech Pl,40.695065,-73.987167,27631,Customer,1982,1,1383,single_trip,one-way,7.0
4,316,2019-12-01 00:06,2019-12-01 00:11,361,Allen St & Hester St,40.716059,-73.991908,307,Canal St & Rutgers St,40.714275,-73.989900,21186,Customer,1975,1,1388,single_trip,one-way,5.0


In [18]:
import pickle

data.to_pickle('../data/data_prepare.pkl')

### Referencias:
https://www.citibikenyc.com/system-data